In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings 
warnings.filterwarnings('ignore')

In [4]:
fake_df=pd.read_csv("Fake.csv")

In [5]:
fake_df["subject"].value_counts()

subject
News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: count, dtype: int64

In [6]:
len(fake_df)

23481

In [7]:
true_df=pd.read_csv("True.csv")

In [8]:
true_df["class"]=1

In [9]:
fake_df["class"]=0

In [10]:
news_df=pd.concat((true_df,fake_df))

In [11]:
type(news_df)

pandas.core.frame.DataFrame

In [12]:
# news_df=news_df.drop(columns=["title","subject"	,"date"])
# # news_df = news_df.drop(columns=["title", "subject", "date"])


In [13]:
news_df

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [14]:
news_df = news_df[['text', 'class']]

In [15]:
news_df

,text,class
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
...,...,...
23476,21st Century Wire says As 21WIRE reported earl...,0
23477,21st Century Wire says It s a familiar theme. ...,0
23478,Patrick Henningsen 21st Century WireRemember ...,0
23479,21st Century Wire says Al Jazeera America will...,0


In [16]:
news_df.isnull().value_counts()

text   class
False  False    44898
Name: count, dtype: int64

In [17]:
# sample(frac=1) → takes 100% of rows but in random order
# random_state=42 → same shuffle every time (good for ML)

In [18]:
news_df=news_df.sample(frac=1,random_state=42).reset_index(drop=True)

In [19]:
news_df

,text,class
0,"Donald Trump s White House is in chaos, and th...",0
1,Now that Donald Trump is the presumptive GOP n...,0
2,Mike Pence is a huge homophobe. He supports ex...,0
3,SAN FRANCISCO (Reuters) - California Attorney ...,1
4,Twisted reasoning is all that comes from Pelos...,0
...,...,...
44893,ABUJA (Reuters) - The United States has formal...,1
44894,Tune in to the Alternate Current Radio Network...,0
44895,I m convinced the Freedom From Religion group...,0
44896,WASHINGTON (Reuters) - The Republican tax plan...,1


In [25]:
import re
import string

In [26]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [27]:
news_df["text"] = news_df["text"].apply(wordopt)

In [38]:
X=news_df['text']
y=news_df['class']

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

In [47]:
len(X_train)

33673

In [49]:
len(X_test)

11225

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
xv_train = tf.fit_transform(X_train)
xv_test = tf.transform(X_test)


In [52]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(xv_train, y_train)


MultinomialNB()

In [53]:
y_pred = model.predict(xv_test)


In [54]:
y_pred

array([1, 0, 0, ..., 1, 0, 0])

In [55]:
from sklearn.metrics import accuracy_score


In [56]:
score=accuracy_score(y_test, y_pred)

In [57]:
score

0.9341648106904231

In [58]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(xv_train, y_train)

y_pred_lr = lr.predict(xv_test)


In [60]:
p=accuracy_score(y_test, y_pred_lr)


In [61]:
p

0.9853006681514477

In [63]:
n = ["Government officially announces new tax policy today",
    "Shocking truth the media does not want you to know",
    "NASA confirms water found on Mars after years of research"]
newprediction = tf.transform(n)
model.predict(newprediction)


array([1, 0, 0])

In [65]:
import pickle

In [68]:
pickle.dump(model, open("fake_news_model.pkl", "wb"))


In [69]:
pickle.dump(tf, open("tfidf_vectorizer.pkl", "wb"))